In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.integrate import odeint

# Определяем систему уравнений Лоренца
def lorenz(state, t, sigma=10.0, beta=8.0/3.0, rho=28.0):
    x, y, z = state
    dxdt = sigma * (y - x)
    dydt = x * (rho - z) - y
    dzdt = x * y - beta * z
    return [dxdt, dydt, dzdt]

# Временной интервал интегрирования
t = np.linspace(0, 50, 10000)

# Начальные условия
initial_state = [1.0, 1.0, 1.0]

# Интегрируем систему уравнений
trajectory = odeint(lorenz, initial_state, t)

# Построение графика аттрактора Лоренца
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot(trajectory[:, 0], trajectory[:, 1], trajectory[:, 2], lw=0.5)
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
plt.title("Аттрактор Лоренца")
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

# Определяем систему уравнений Лоренца
def lorenz(state, t, sigma=10.0, beta=8.0/3.0, rho=28.0):
    x, y, z = state
    dxdt = sigma * (y - x)
    dydt = x * (rho - z) - y
    dzdt = x * y - beta * z
    return [dxdt, dydt, dzdt]

# Временной интервал интегрирования
t = np.linspace(0, 500, 100000)

# Начальные условия
initial_state = [1.0, 1.0, 1.0]

# Интегрируем систему уравнений
trajectory = odeint(lorenz, initial_state, t)

# Построение двумерной проекции аттрактора Лоренца на плоскости xy
plt.figure(figsize=(10, 6))
plt.plot(trajectory[:, 0], trajectory[:, 1], lw=0.5)
plt.xlabel("X")
plt.ylabel("Y")
plt.title("Двумерная проекция аттрактора Лоренца (xy)")
plt.grid(True)
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from scipy.stats import gaussian_kde

# Определяем систему уравнений Лоренца
def lorenz(state, t, sigma=10.0, beta=8.0/3.0, rho=28.0):
    x, y, z = state
    dxdt = sigma * (y - x)
    dydt = x * (rho - z) - y
    dzdt = x * y - beta * z
    return [dxdt, dydt, dzdt]

# Увеличенный временной интервал
t = np.linspace(0, 500, 100000)
initial_state = [1.0, 1.0, 1.0]

# Интегрируем систему
trajectory = odeint(lorenz, initial_state, t)
x = trajectory[:, 0]
y = trajectory[:, 1]

# Вычисляем плотность точек по (x, y)
xy = np.vstack([x, y])
density = gaussian_kde(xy)(xy)

# Построение графика: каждая точка раскрашивается согласно локальной плотности
plt.figure(figsize=(10, 6))
sc = plt.scatter(x, y, c=density, s=0.5, cmap='hot', marker='.', linewidth=0)
plt.xlabel("X")
plt.ylabel("Y")
plt.title("Двумерная проекция аттрактора Лоренца с яркостью по плотности")
plt.colorbar(sc, label='Локальная плотность')
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

def trajectory_to_image(x, y, img_size=100, light_gray=200):
    """
    Принимает координаты траектории (x, y) и создает изображение размером img_size x img_size,
    где:
      - пиксели, в которые не попадала траектория, белые (255);
      - если в пиксель попала точка ровно 1 раз, его серый цвет равен light_gray (например, 200);
      - пиксели с максимальным числом попаданий становятся чёрными (0);
      - для остальных используется линейный градиент между light_gray и 0.
    """
    # Определяем диапазоны координат
    x_min, x_max = np.min(x), np.max(x)
    y_min, y_max = np.min(y), np.max(y)
    
    # Инициализируем матрицу подсчёта попаданий (плотности) размером img_size x img_size
    density_matrix = np.zeros((img_size, img_size), dtype=np.int32)
    
    # Проходим по всем точкам траектории и определяем, в какой пиксель попадает каждая точка
    for xi, yi in zip(x, y):
        # Преобразуем координаты в индексы от 0 до img_size-1
        # Используем int() (это аналог np.floor, если число положительное)
        ix = int((xi - x_min) / (x_max - x_min) * (img_size - 1))
        iy = int((yi - y_min) / (y_max - y_min) * (img_size - 1))
        density_matrix[iy, ix] += 1

    # Определяем максимальное число попаданий (игнорируя ячейки с 0)
    if np.any(density_matrix > 0):
        max_count = density_matrix[density_matrix > 0].max()
    else:
        max_count = 1  # на случай, если ни одна точка не попала (не должно случаться)

    # Создаем итоговое изображение: изначально все пиксели белые (255)
    img = np.full((img_size, img_size), 255, dtype=np.uint8)
    
    # Для ячеек, в которые попала траектория, задаем интенсивность по градиенту:
    # если попадание = 1, то интенсивность = light_gray (светло-серый),
    # если попадание = max_count, то интенсивность = 0 (чёрный);
    # для промежуточных значений выполняется линейная интерполяция.
    mask = density_matrix > 0
    if max_count == 1:
        # Если максимум равен 1, то во всех заполненных ячейках назначаем light_gray
        img[mask] = light_gray
    else:
        # Линейное преобразование: для count c (c>=1)
        # intensity = light_gray - (light_gray * (c - 1) / (max_count - 1))
        #intensities = light_gray - (light_gray * (density_matrix[mask] - 1) / max_count)
        intensities = light_gray  * pow(1 - density_matrix[mask] / max_count, 20)
        # Гарантируем, что значение находится в диапазоне 0...255
        intensities = np.clip(intensities, 0, 255)
        img[mask] = intensities.astype(np.uint8)
    
    return img

# -------------------------
# Пример: аттрактор Лоренца и построение его проекции на плоскость xy
# -------------------------

def lorenz(state, t, sigma=10.0, beta=8.0/3.0, rho=28.0):
    x, y, z = state
    dxdt = sigma * (y - x)
    dydt = x * (rho - z) - y
    dzdt = x * y - beta * z
    return [dxdt, dydt, dzdt]

# Задаем временной интервал и интегрируем систему
t = np.linspace(0, 5000, 1000000)
initial_state = [1.0, 1.0, 1.0]
trajectory = odeint(lorenz, initial_state, t)
x = trajectory[:, 0]
y = trajectory[:, 1]

# Получаем изображение 100x100 по траектории
img = trajectory_to_image(x, y, img_size=200, light_gray=200)

# Отображаем полученное изображение
plt.figure(figsize=(6, 6))
# Параметр origin='lower' задаёт нижнюю левую точку в качестве начала координат,
# extent позволяет масштабировать оси в соответствии с исходными координатами
plt.imshow(img, cmap='gray', origin='lower',
           extent=[np.min(x), np.max(x), np.min(y), np.max(y)])
plt.xlabel("X")
plt.ylabel("Y")
plt.title("Двумерная проекция аттрактора Лоренца")
plt.show()
